<a href="https://colab.research.google.com/github/rajy4683/EVA4S4/blob/master/S4EVA4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
!pip install torchsummary
from torchsummary import summary

In [0]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1) #input -? OUtput? RF
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv5 = nn.Conv2d(256, 512, 3)
        self.conv6 = nn.Conv2d(512, 1024, 3)
        self.conv7 = nn.Conv2d(1024, 10, 3)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
        x = F.relu(self.conv6(F.relu(self.conv5(x))))
        x = F.relu(self.conv7(x))
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [0]:
class Net(nn.Module):
    def __init__(self,start_channels=32, exponetate_layers=True):
        super(Net, self).__init__()
        self.start_channels = start_channels
        self.multiplier = 2

        if (exponetate_layers == False):
            self.multiplier = 1
        
        self.conv1 = nn.Conv2d(1, self.start_channels, 3, padding=1) #input -? OUtput? RF
        self.conv1_bn = nn.BatchNorm2d(self.start_channels)
        self.conv2 = nn.Conv2d(self.start_channels, self.start_channels*self.multiplier, 3, padding=1)
        self.conv2_bn = nn.BatchNorm2d(self.start_channels*self.multiplier)

        self.start_channels = self.start_channels * self.multiplier
        self.pool1 = nn.MaxPool2d(2, 2)

        self.conv3 = nn.Conv2d(self.start_channels, self.start_channels*self.multiplier, 3, padding=1)
        self.conv3_bn = nn.BatchNorm2d(self.start_channels*self.multiplier)
        self.start_channels = self.start_channels * self.multiplier
        self.conv4 = nn.Conv2d(self.start_channels, self.start_channels*self.multiplier, 3, padding=1)
        self.conv4_bn = nn.BatchNorm2d(self.start_channels*self.multiplier)
        self.start_channels = self.start_channels * self.multiplier
        self.pool2 = nn.MaxPool2d(2, 2)

        self.conv5 = nn.Conv2d(self.start_channels, self.start_channels*self.multiplier, 3)
        self.conv5_bn = nn.BatchNorm2d(self.start_channels*self.multiplier)
        self.start_channels = self.start_channels * self.multiplier
        self.conv6 = nn.Conv2d(self.start_channels, self.start_channels*self.multiplier, 3)
        self.conv6_bn = nn.BatchNorm2d(self.start_channels*self.multiplier)
        self.start_channels = self.start_channels * self.multiplier

        self.conv7 = nn.Conv2d(self.start_channels, 10, 3)

    def forward(self, x):
        x = F.relu(self.conv1_bn(self.conv1(x))) ### Layer 1 ---->(Relu(BN(Conv)))
        x = self.pool1(F.relu(self.conv2_bn(self.conv2(x)))) ### Layer 2---> MaxPool((Relu(BN(Conv))))
        x = F.relu(self.conv3_bn(self.conv3(x))) ### Layer 3 ---->(Relu(BN(Conv)))
        x = self.pool2(F.relu(self.conv4_bn(self.conv4(x)))) ### Layer 4---> MaxPool((Relu(BN(Conv))))
        #########
        x = F.relu(self.conv5_bn(self.conv5(x))) ### Layer 5---> (Relu(BN(Conv)))
        x = F.relu(self.conv6_bn(self.conv6(x))) ### Layer 6 ---> (Relu(BN(Conv)))
        x = self.conv7(x) #### Final Layer (BN(Conv)))
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [13]:
model = Net(start_channels=16,exponetate_layers=False).to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 28, 28]             160
       BatchNorm2d-2           [-1, 16, 28, 28]              32
            Conv2d-3           [-1, 16, 28, 28]           2,320
       BatchNorm2d-4           [-1, 16, 28, 28]              32
         MaxPool2d-5           [-1, 16, 14, 14]               0
            Conv2d-6           [-1, 16, 14, 14]           2,320
       BatchNorm2d-7           [-1, 16, 14, 14]              32
            Conv2d-8           [-1, 16, 14, 14]           2,320
       BatchNorm2d-9           [-1, 16, 14, 14]              32
        MaxPool2d-10             [-1, 16, 7, 7]               0
           Conv2d-11             [-1, 16, 5, 5]           2,320
      BatchNorm2d-12             [-1, 16, 5, 5]              32
           Conv2d-13             [-1, 16, 3, 3]           2,320
      BatchNorm2d-14             [-1, 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:45: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [0]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [15]:
train_loader.dataset.data.shape

torch.Size([60000, 28, 28])

In [0]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nEpoch: {:.0f} Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        epoch, test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [17]:

model = Net(start_channels=16,exponetate_layers=False).to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 20):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]


Epoch: 1 Test set: Average loss: 0.0480, Accuracy: 9850/10000 (98.50%)



  0%|          | 0/469 [00:00<?, ?it/s]


Epoch: 2 Test set: Average loss: 0.0445, Accuracy: 9859/10000 (98.59%)



  0%|          | 0/469 [00:00<?, ?it/s]

...


Epoch: 17 Test set: Average loss: 0.0189, Accuracy: 9933/10000 (99.33%)



  0%|          | 0/469 [00:00<?, ?it/s]


Epoch: 18 Test set: Average loss: 0.0182, Accuracy: 9943/10000 (99.43%)



loss=0.0006617109174840152 batch_id=468: 100%|██████████| 469/469 [00:18<00:00, 25.95it/s]



Epoch: 19 Test set: Average loss: 0.0183, Accuracy: 9948/10000 (99.48%)

